# ME449 - Homework 3 - Zhengyang Kris Weng Submission

In [1]:
import numpy as np
import modern_robotics as mr
from tqdm import tqdm
import ur5_parameters
ur5 = ur5_parameters.UR5()

## Part0: Puppet function

## Part 1: Simulating a falling robot. 
In the first part, the robot will fall in gravity without
damping or the external spring (joint damping and spring stiffness are set to zero). Since there is
no damping or friction, the total energy of the robot (kinetic plus potential) should remain constant
during motion. Gravity is g = 9.81 m/s2 in the −ˆzs-direction, i.e., gravity acts downward.
Simulate the robot falling from rest at the home configuration for five seconds. The output data
should be saved as a .csv file, where each of the N rows has six numbers separated by commas.
This .csv file is suitable for animation with the CoppeliaSim UR5 csv animation scene. Adjust
the animation scene playback speed (“Time Multiplier”) so it takes roughly five seconds of wall
clock time to play your csv file. You can evaluate if your simulation is preserving total energy by
visually checking if the robot appears to swing to the same height (same potential energy) each
swing. Choose values of dt (a) where the energy appears nearly constant (without choosing dt
unnecessarily small) and (b) where the energy does not appear constant (because your timestep
is too coarse). Capture a video for each case and note the dt chosen for each case. Explain how
you would calculate the total energy of the robot at each timestep if you wanted to plot the total
energy to confirm that your simulation approximately preserves it.

In [2]:
# Puppet function:
def puppet_q1(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q1: free falling in gravity

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, np.zeros(n), g, np.zeros(n), Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [29]:
q1_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q1_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])

q1_thetamat, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.001, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat.csv', q1_thetamat, delimiter=',')

100%|██████████| 5000/5000 [00:34<00:00, 143.02it/s]


In [30]:
q1_thetamat_coarse, _ = puppet_q1(q1_thetalist0, q1_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 0, 0, 0)

# Save to csv file
np.savetxt('q1_thetamat_coarse.csv', q1_thetamat_coarse, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 136.09it/s]


Loooking at video part1a and part1b.mp4, we can see that energy is not conserved due to error in numerical integration when timestep is too coarse. A good measure of total system energy would be the Hamiltonian of the system, as a sum of potential energy and kinetic energy of each link. By plotting the trend of Hamiltonian of each configuration in the trajectory, this will visualize the preservation of energy in the system. [TODO: IMPLEMENT COMPUTE H WHEN HAVE TIME]

## Part 2: Adding damping. 
Now experiment with different damping coefficients as the robot
falls from the home configuration. Damping causes a torque at each joint equal to the negative of
the joint rate times the damping. Create two videos showing that (a) when you choose damping to
be positive, the robot loses energy as it swings, and (b) when you choose damping to be negative,
the robot gains energy as it swings. Use t = 5 s and dt = 0.01 s, and for the case of positive
damping, the damping coefficient should almost (but not quite) bring the robot to rest by the end
of the video. Do you see any strange behavior in the simulation if you choose the damping constant
to be a large positive value? Can you explain it? How would this behavior change if you chose
shorter simulation timesteps?

In [3]:
def puppet_q2(thetalist, dthetalist, g, Mlist, Slist, Glist, t, dt, damping, stiffness, restLength):
    """
    Simulate a robot under damping and spring reaction. Q2: Adding damping to robot. Damping causes a tau = - dtheta * damping

    Args:
        thetalist (np.array): n-vector of initial joint angles (rad)
        dthetalist (np.array): n-vector of initial joint velocities (rad/s)
        g (np.array): 3-vector of gravity in s frame (m/s^2)
        Mlist (np.array): 8 frames of link configuration at home pose
        Slist (np.array): 6-vector of screw axes at home configuration
        Glist (np.array): Spatial inertia matrices of the links
        t (float): total simulation time (s)
        dt (float): simulation time step (s)
        damping (float): viscous damping coefficient (Nmn/rad)
        stiffness (float): spring stiffness coefficient (N/m)
        restLength (float): length of the spring at rest (m)
    Returns:
        thetamat (np.array): N x n matrix of joint angles (rad). Each row is a set of joint angles
        dthetamat (np.array): N x n matrix of joint velocities (rad/s). Each row is a set of joint velocities
    """
    # Initialize 
    N = int(t/dt)
    n = len(thetalist)
    thetamat = np.zeros((N + 1, n))
    dthetamat = np.zeros((N + 1, n))
    thetamat[0] = thetalist
    dthetamat[0] = dthetalist

    for i in tqdm(range(N)):
        tau_damping = - damping * dthetalist
        i_acc = mr.ForwardDynamics(thetalist, dthetalist, tau_damping, g, np.zeros(n), Mlist, Glist, Slist) 
        i_pos, i_vel = mr.EulerStep(thetalist, dthetalist, i_acc, dt)
        thetamat[i + 1] = i_pos
        dthetamat[i + 1] = i_vel

        # Update
        thetalist = i_pos
        dthetalist = i_vel
    
    return thetamat, dthetamat

In [7]:
q2_thetalist0 = np.array([0, 0, 0, 0, 0, 0])
q2_dthetalist0 = np.array([0, 0, 0, 0, 0, 0])
g = np.array([0, 0, -9.81])

q2_thetamat, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 1.2, 0, 0)
# Save to csv file
np.savetxt('q2_thetamat.csv', q2_thetamat, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 153.18it/s]


In [18]:
q2_thetamat_neg, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, -0.02, 0, 0)
# Save to csv file
print(q2_thetamat_neg)
np.savetxt('q2_thetamat_neg.csv', q2_thetamat_neg, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 135.91it/s]

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.57237340e-03 -2.87368129e-03  3.01307887e-04
   3.18411925e-22 -1.13423177e-18]
 ...
 [-3.07812861e-01 -9.92195181e+00  1.31700123e+01 -6.81968171e+00
  -8.36251555e-01  5.89264993e+01]
 [-3.02987560e-01 -1.00110250e+01  1.32499198e+01 -6.84260777e+00
  -8.76917115e-01  5.96387109e+01]
 [-2.97193787e-01 -1.00998345e+01  1.33216146e+01 -6.85772600e+00
  -9.17977817e-01  6.03581957e+01]]


In [ ]:
q2_thetamat_large, _ = puppet_q2(q2_thetalist0, q2_dthetalist0, g, ur5.Mlist, ur5.Slist, ur5.Glist, 5, 0.01, 20, 0, 0)
# Save to csv file
print(q2_thetamat_large)
np.savetxt('q2_thetamat_large.csv', q2_thetamat_large, delimiter=',')

100%|██████████| 500/500 [00:03<00:00, 141.24it/s]

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.57237340e-03 -2.87368129e-03  3.01307887e-04
   3.18411925e-22 -1.13423177e-18]
 ...
 [            nan             nan             nan             nan
              nan             nan]
 [            nan             nan             nan             nan
              nan             nan]
 [            nan             nan             nan             nan
              nan             nan]]


As I increase the magnitude of damping coefficient in the simulation, the function start to run into numerical stability issues and starts to produce nan values in output. This happend because of the numerical instability in the euler integration and the fact that coarse timestep landing next iteration on gradients amplifying such effect; as a result simulation output grew at an extremely fast rate and overflowed. Increasing granularity in timestep helps addressing this issue.